In [ ]:
import pandas as pd
from transformers import AutoTokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict  # Import Dataset and DatasetDict classes

# Load the dataset
dataset_path = r"C:\Users\mouni\Downloads\archive\Arabic_Poetry_Dataset.csv"
df = pd.read_csv(dataset_path)
# Preprocess the 'poem_count' column
df['poem_count'] = df['poem_count'].str.extract(r'(\d+)')
df = df.dropna(subset=['poem_count'])
df['poem_count'] = df['poem_count'].astype(int)
df = df[df['poem_count'] <= 30]
# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained("aubmindlab/aragpt2-base")

# Set the pad_token to eos_token
tokenizer.pad_token = tokenizer.eos_token

# Step 1: Split the data into training and test sets
train_df = df.sample(frac=0.9, random_state=42)
test_df = df.drop(train_df.index)

# Step 2: Keep only the specified columns
columns_to_keep = ['poem_text', 'poem_title', 'poem_count']
train_df = train_df[columns_to_keep]
test_df = test_df[columns_to_keep]

# Step 3: Combine the columns into a single column to create a well-formed prompt
def create_prompt(row):
    return f"القصيدة تحت عنوان {row['poem_title']}:\n{row['poem_text']}"

train_df['prompt'] = train_df.apply(create_prompt, axis=1)
test_df['prompt'] = test_df.apply(create_prompt, axis=1)

# Step 4: Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['prompt'], truncation=True, max_length=200, padding='max_length')

train_dataset = Dataset.from_pandas(train_df[['prompt']])
test_dataset = Dataset.from_pandas(test_df[['prompt']])

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

# Create a DatasetDict
dataset_dict = DatasetDict({
    'train': tokenized_train_dataset,
    'test': tokenized_test_dataset
})

Map:   0%|          | 0/59937 [00:00<?, ? examples/s]

Map:   0%|          | 0/6660 [00:00<?, ? examples/s]

In [4]:
print(train_dataset[1])

{'prompt': 'القصيدة تحت عنوان قل لابن بوران إن كان ابن بوران:\nقُلْ لابن بورانَ إن كان ابنَ بوران\nفإنَّ شكّيَ فيه جُلُّ إيماني\nيا باطلاً أوهمْتنيه مَخايلُه\nبلا دليل ولا تثبيتِ بُرهانِ\nما أنتَ إلا خيالٌ طاف طائفه\nوما هجائيك إلا هجْرُ وسْنان\nقد كنتُ أحسبه شيئاً فأهجوه\nحتى أزاح يقيني فيه حسباني\n', '__index_level_0__': 48430}


In [2]:
from transformers import AutoTokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling

# Initialize model
model_name = "aubmindlab/aragpt2-base"
model = GPT2LMHeadModel.from_pretrained(model_name)

# Initialize Data Collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

# Training arguments
training_args = TrainingArguments(
    output_dir="./output",
    overwrite_output_dir=True,
    num_train_epochs=8,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    evaluation_strategy="no",  # Set evaluation strategy to "no"
    save_strategy="epoch",
    logging_dir="./logs",
    save_total_limit=2,
    fp16=True,  # Enable mixed precision
    learning_rate=5e-5,
    weight_decay=0.01,
    report_to="none"
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_dict['train'],
    eval_dataset=dataset_dict['test'],
    data_collator=data_collator
)

# Start training
trainer.train()

c:\Users\mouni\anaconda3\Lib\site-packages\transformers\training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/59936 [00:00<?, ?it/s]

{'loss': 15.035, 'grad_norm': 8.02468490600586, 'learning_rate': 4.958705953016551e-05, 'epoch': 0.07}
{'loss': 11.8577, 'grad_norm': 8.912354469299316, 'learning_rate': 4.9169947944474107e-05, 'epoch': 0.13}
{'loss': 11.0717, 'grad_norm': 7.924450397491455, 'learning_rate': 4.87528363587827e-05, 'epoch': 0.2}
{'loss': 10.659, 'grad_norm': 7.131441593170166, 'learning_rate': 4.83357247730913e-05, 'epoch': 0.27}
{'loss': 10.2303, 'grad_norm': 7.8812994956970215, 'learning_rate': 4.7918613187399895e-05, 'epoch': 0.33}
{'loss': 10.0629, 'grad_norm': 7.4868903160095215, 'learning_rate': 4.750150160170849e-05, 'epoch': 0.4}
{'loss': 10.0251, 'grad_norm': 6.540153980255127, 'learning_rate': 4.708439001601709e-05, 'epoch': 0.47}
{'loss': 9.7642, 'grad_norm': 8.012529373168945, 'learning_rate': 4.666727843032568e-05, 'epoch': 0.53}
{'loss': 9.8018, 'grad_norm': 6.098832607269287, 'learning_rate': 4.625016684463428e-05, 'epoch': 0.6}
{'loss': 9.6032, 'grad_norm': 6.378545761108398, 'learning_ra

TrainOutput(global_step=59936, training_loss=8.202004232574643, metrics={'train_runtime': 30190.6925, 'train_samples_per_second': 15.882, 'train_steps_per_second': 1.985, 'total_flos': 4.89392831232e+16, 'train_loss': 8.202004232574643, 'epoch': 7.999733057492742})

In [57]:
# Save the trained model
trainer.save_model("./output/final_model")  # Save the final model to the specified path
tokenizer.save_pretrained("./output/final_model")  # Save tokenizer if needed

('./output/final_model\\tokenizer_config.json',
 './output/final_model\\special_tokens_map.json',
 './output/final_model\\vocab.json',
 './output/final_model\\merges.txt',
 './output/final_model\\added_tokens.json',
 './output/final_model\\tokenizer.json')

In [58]:
model = GPT2LMHeadModel.from_pretrained("./output/final_model")


In [8]:
# Evaluate the model
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

  0%|          | 0/833 [00:00<?, ?it/s]

Evaluation results: {'eval_loss': 2.014402389526367, 'eval_runtime': 239.4681, 'eval_samples_per_second': 27.812, 'eval_steps_per_second': 3.479, 'epoch': 7.999733057492742}


In [ ]:
"""from transformers import  DataCollatorForLanguageModeling

# Initialize model
model_name = "aubmindlab/aragpt2-large"
model = GPT2LMHeadModel.from_pretrained(model_name)

# Training arguments
training_args = TrainingArguments(
    output_dir="./output",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    save_total_limit=2,
    fp16=True,
    learning_rate=5e-5,
    weight_decay=0.01,
    report_to="none"
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_dict['train'],
    eval_dataset=dataset_dict['test'],
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
)

# Start training
trainer.train()
"""

In [55]:
import torch

# Function to generate text using the model with top-k and top-p sampling
def generate_text(prompt, max_length=150, num_return_sequences=1, top_k=30, top_p=0.7):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_length=max_length + len(inputs['input_ids'][0]),  # Adjust max_length to account for the prompt length
        num_return_sequences=num_return_sequences,
        do_sample=True,  # Enable sampling
        top_k=top_k,     # Top-k sampling
        top_p=top_p,     # Top-p (nucleus) sampling
        temperature=0.2,  # Adjust temperature for more randomness
        # pad_token_id=50256,
        repetition_penalty=1.5,
    )
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text[len(prompt):].strip()  # Remove the prompt from the generated text

# Example prompts
prompts = [
     "أكتب بيتًا شعريًا بأسلوب عربي فصيح يعبر عن ألم الخيانة والفراق بين الحبيبة وحبيبها، مستخدمًا كلمات مثل: خيانة، وجع، دمع، شوق، فراق، وداع. يجب أن يكون البيت مليئًا بالعاطفة ويعبر عن الحزن العميق."
    #  "أكتب قصيدة عن حب الوطن"
]



# Generate and print text for each prompt
for prompt in prompts:
    generated_text = generate_text(prompt)
    print(f"Prompt: {prompt}")
    print(f"Generated: {generated_text}")
    print("\n")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Prompt: أكتب بيتًا شعريًا بأسلوب عربي فصيح يعبر عن ألم الخيانة والفراق بين الحبيبة وحبيبها، مستخدمًا كلمات مثل: خيانة، وجع، دمع، شوق، فراق، وداع. يجب أن يكون البيت مليئًا بالعاطفة ويعبر عن الحزن العميق.
Generated: وإني لأعلم ما قد كنت أعلم أنني رجلُهوى وإننى ال يدى اللوى بها الدلى
وأرى كل من رامى الشعر في الناس كلهم عدواَ ولا عدوةٍ له شرف الغنى
فلا زلت ذا أدب لا يزال مع الأيام طالبال به العلم والتقى
ولا زال هذا الدهر حتى صار آخر أيامه مقيماّةِ الحشا متهللاـعبةً
على أنه كان لي قبل اليوم خير عيش وأوطار ومنزلْتضى العمر والعطب معاً
وما أنا إلا العيش الذي لم يزل عندي غير أهله طامعي الهوى
ولكن أرى نفسي بعد الموت أقرب منها إلى قرب منزلٍ بعيدتي
وقد أصبحت فيما


